## FM层

![avatar](FM.png)

首先讲一下为什么这个网络结构的结果是FM的结果  
对于经典的FM，其公式是
$$y = \sum_{i=1}^{n}w_ix_i+\sum_{i=1}^{n}\sum_{j=i+1}^{n}<v_i,v_j>x_ix_j\tag{1}$$  
在deepFM中，通过FM来实现 embedding层，这里每个Field是类别特征onehot之后的结果

例如一个类别特征是$\left[\begin{matrix} 1 \\ 3 \\ 0 \end{matrix}\right]$,它的onehot编码结果为$\left[\begin{matrix} 0&1&0&0 \\ 0&0&0&1 \\ 1&0&0&0 \end{matrix}\right]$


In [1]:
from mxnet import nd
X = nd.array([[1],[3],[0]])
X_onehot = nd.one_hot(X[:,0],depth = int(nd.max(X[:,0]).asscalar()+1))
X_onehot


[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]
<NDArray 3x4 @cpu(0)>

现在来看$field_i$ 和$filed_j$在这里的计算情况，假设$fild_j$有$f_i$维，$field_j$有$f_j$维，那么可以设$feild_i$的数据为$X_{n \times field_j}$,$feild_i$的数据为$X_{n \times field_j}$,要将$X_{n \times field_j}$映射为$k$的隐dense向量，可以$X_{n \times field_i}V_{field_i \times k}$。
$$
V_{field_i \times k} = 
\left[\begin{matrix}v_{11} & v_{12} & \cdots & v_{1k} \\
v_{21} & v_{22} & \cdots & v_{2k}\\
\cdots & \cdots & \cdots & \cdots\\
v_{{field_i}1} & v_{{field_i}2} & \cdots & v_{{field_i}k}
\end{matrix}
\right]
$$  

对于数据$
\left[\begin{matrix}x_{11} & x_{12} & \cdots & x_{1{field_i}} \\
\end{matrix}
\right]$,它除了某一项$x_1m=1$以外，别的项$x_{1k_{(k \ne m)}}=0$，当它和$V$点乘时有
$$
x_1V = 
\left[\begin{matrix}x_{11} & x_{12} & \cdots & x_{1{field_i}} \\
\end{matrix}
\right]
\left[\begin{matrix}v_{11} & v_{12} & \cdots & v_{1k} \\
v_{21} & v_{22} & \cdots & v_{2k}\\
\cdots & \cdots & \cdots & \cdots\\
v_{{field_i}1} & v_{{field_i}2} & \cdots & v_{{field_i}k}
\end{matrix}
\right]
=
\left[\begin{matrix}v_{m1} & v_{m2} & \cdots & v_{mk} \\
\end{matrix}
\right] x_{1m}
=v_i x_{1m}
$$

同理，对于$field_j$有，假设$x_1n=1$
$$
x_1V = 
\left[\begin{matrix}x_{11} & x_{12} & \cdots & x_{1{field_j}} \\
\end{matrix}
\right]
\left[\begin{matrix}v_{11} & v_{12} & \cdots & v_{1k} \\
v_{21} & v_{22} & \cdots & v_{2k}\\
\cdots & \cdots & \cdots & \cdots\\
v_{{field_j}1} & v_{{field_j}2} & \cdots & v_{{field_j}k}
\end{matrix}
\right]
=
\left[\begin{matrix}v_{n1} & v_{n2} & \cdots & v_{nk} \\
\end{matrix} 
\right] x_{1n}
= v_j x_{1n} 
$$

也就是这两个$field$的dense embedding的结果为
$$
dense_i = 
\left[\begin{matrix}v_{m1} & v_{m2} & \cdots & v_{mk} \\
\end{matrix}
\right] x_{1m}
=v_i x_{1m}
$$
$$
dense_j =
\left[\begin{matrix}v_{n1} & v_{n2} & \cdots & v_{nk} \\
\end{matrix}
\right] x_{1n}
= v_j x_{1n}
$$
在FM Layer里，这些dense embeddings的结果做inner product，也就是$dense_i$和$dense_j$做向量内积。
$$dense_i dense_j = <v_i,v_j>x_{1m}x_{1n}
$$
这正是$(1)$中的二阶项，而对于$i \ne m$和$j \ne n$，有$x_{1i}=0,x_{1j}=0$，二阶项为0。  


### 可以看到这里的dense embeddings的结果也就是FM模型中的隐向量

那么具体如何实现呢，从前面的分析可以知道，对单个$field$（分类变量one-hot的结果）的dense embeddings的结果就是隐向量矩阵$V$的某个行向量，也就是说，对于训练数据$X_1$,它dense embeddings的结果就是1所在的列号，在$V$中对应的行号的行向量的拼接的结果；下面提供了两种方法来实现。我们确保了他们的结果是一致的，并且在计算图中的求导也是一致的。

首先生成数据，这是有一个500维的分类数据和1000维的分类数据组成，进行onehot编码

In [1]:
from mxnet.gluon import nn
from mxnet import nd, autograd
import mxnet as mx
import numpy as np
x1 = np.random.randint(0,500, size = (100000,1))
x2 = np.random.randint(0,1000, size = (100000,1))
X = nd.array(np.concatenate((x1,x2), axis = 1))
X_onehot1 = nd.one_hot(X[:,0],depth = int(nd.max(X[:,0]).asscalar()+1))
X_onehot2 = nd.one_hot(X[:,1],depth = int(nd.max(X[:,1]).asscalar()+1))
X = nd.concat(X_onehot1,X_onehot2,dim = 1)

生成隐向量矩阵$V$

In [2]:
v = nd.random.randn(1500, 5)

In [3]:
v1 = nd.array(v)

In [4]:
id(v1)==id(v)

False

In [5]:
v1.attach_grad()

In [6]:
v.attach_grad()

In [7]:
v.grad, v1.grad

(
 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]
 <NDArray 1500x5 @cpu(0)>, 
 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]
 <NDArray 1500x5 @cpu(0)>)

方法一、使用take去取得1对应列号在$V$中行号的隐向量，再拉伸成embeddings层的维度

In [8]:
with autograd.record():
    emb = nd.take(v, nd.array(np.argwhere(X.asnumpy())[:,1].reshape(-1,2))).reshape(-1, 5*2)
    y = nd.sum(emb, axis=1)

In [9]:
y


[-2.7958186   2.606924    0.87531114 ...  3.561966   -3.6046371
  4.7082334 ]
<NDArray 100000 @cpu(0)>

In [10]:
y.backward()

In [11]:
v.grad


[[218. 218. 218. 218. 218.]
 [198. 198. 198. 198. 198.]
 [205. 205. 205. 205. 205.]
 ...
 [106. 106. 106. 106. 106.]
 [ 87.  87.  87.  87.  87.]
 [104. 104. 104. 104. 104.]]
<NDArray 1500x5 @cpu(0)>

方法二、使用直接计算的方法，要注意的是对每个$field$要单独计算，最后在讲embeddings拼接起来

In [12]:
with autograd.record():
    t = nd.concat(nd.dot(X[:,:500],v1[:500,:]),nd.dot(X[:,500:],v1[500:,:]), dim=1)
    y2 = nd.sum(t, axis=1)

In [13]:
y2


[-2.7958186   2.606924    0.87531114 ...  3.561966   -3.6046371
  4.7082334 ]
<NDArray 100000 @cpu(0)>

In [14]:
y2.backward()

In [15]:
v1.grad


[[218. 218. 218. 218. 218.]
 [198. 198. 198. 198. 198.]
 [205. 205. 205. 205. 205.]
 ...
 [106. 106. 106. 106. 106.]
 [ 87.  87.  87.  87.  87.]
 [104. 104. 104. 104. 104.]]
<NDArray 1500x5 @cpu(0)>

可以看到两种方法是一致的

# DNN层

![avatar](DNN.png)